# Домашнее задание 1. Всего понемножку

Дедлайн: 21 декабря 2019 23:59:59

1) Пожалуйста, создайте репозиторий на github.com и сделайте его приватным (https://clck.ru/KMNwC, https://clck.ru/KMNmL) - так вашу работу сможете видеть только вы. Подключитесь к https://education.github.com/pack с помощью edu.hse.ru почты, чтобы можно было добавлять в collaborators неограниченное количество пользователей.

2) Чтобы преподаватели и ассистенты могли просматривать ваш приватный репозиторий, добавьте в collaborators вашего репозитория (https://clck.ru/KMP9Z) четырёх человек: esolovev, eszakharova, akseryanin, ldsad7.

3) Можно сдать как один или несколько .py-файлов, так и .ipynb-тетрадку. Несоблюдение PEP8 пока что не карается.

4) **При написании функций/классов используйте type annotations** (подписывайте типы параметров и выходных значений, каждую используемую переменную подписывать не нужно). Сжато о type annotations рассказал [здесь](https://github.com/esolovev/ling2019/blob/master/module1/some-python-features.ipynb), подробнее читайте в [документации](https://docs.python.org/3/library/typing.html).

5) Нельзя использовать pandas и numpy, а также другие сторонние библиотеки для работы с данными.

## Часть 1. Скачиваем Топ-500 КиноПоиска.

1) (3 балла) Напишите функцию без параметров, возвращающую генератор, который последовательно возвращает информацию о фильмах из [Топ-500 КиноПоиска](https://www.kinopoisk.ru/top/lists/1/) в виде словаря. Например, первый возвращаемый объект будет выглядеть так (достаточно вытащить только перечисленные пять полей): 

`{
    'name': 'Побег из Шоушенка', 
    'kp_rating': 9.112, 
    'imdb_rating': 9.3, 
    'year': 1994, 
    'url': 'https://www.kinopoisk.ru/film/326/',
    '_utc_timestamp': '2019-11-26 09:53:51' # время в UTC (https://en.wikipedia.org/wiki/Coordinated_Universal_Time) на момент загрузки страницы c точностью до секунд
}`

Используйте любые библиотеки для хождения в интернет (я рекомендую [requests](https://2.python-requests.org/en/master/)) и парсинга HTML (например, [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)).

Делайте перерывы в 0.5 секунд между запросами. Это можно сделать с помощью одной из функций в модуле [time](https://docs.python.org/3/library/time.html). Изучите документацию и примените нужную функцию.

Поле `_utc_timestamp` можно сформировать следующим образом: сначала получаем объект с текущей датой и временем, а затем форматируем его как строку. Вам поможет модуль [datetime](https://docs.python.org/3/library/datetime.html). Обратите внимание, что ведущие нули (в моем примере это нуль перед часом - 09) нужно оставлять в строке, то есть строка со временем должна иметь фиксированную длину 19.

**UPD**: если не получается загружать страницы с КП, можно взять в качестве источника данных https://www.kinoafisha.info/rating/movies/?page=2 (в таком случае рейтинг IMDb можно не доставать).

In [0]:
import requests
import re 
import time
import datetime
from bs4 import BeautifulSoup
from typing import List, Dict, Union, Any, Iterable


In [0]:
def kino_gen() -> Iterable[Dict[str, Any]]:
  required_tags = []
  for page in [1,2,3]:
     url = 'https://www.kinopoisk.ru/top/lists/1/filtr/all/sort/order/perpage/200/page/{}/'.format(page)
     headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}  
     r = requests.get(url, headers=headers)
     html_doc = r.text  
     soup = BeautifulSoup(html_doc, 'html.parser')
     required_tags += soup.find_all('tr', ' js-film-list-item')
     time.sleep(10)
  for tr in required_tags:
    td_blocks = tr.find_all('td')
    div_block1 = td_blocks[1].find('div')
    div_block2 = td_blocks[2].find('div', 'WidgetStars')
    divs_in_td2 = td_blocks[2].find_all('div')
    name = div_block1.find('a').contents[0]
    reg_year = re.search(r'(\d+)', div_block1.find('span').contents[0])
    if len(reg_year.group(1)) < 4:
      year = ''
    else:
      year = reg_year.group(1)
    kp_rating = div_block2.get('data-film-rating')
    link = 'https://www.kinopoisk.ru/film/%s/' % div_block2.get('data-film-id')
    imdb = divs_in_td2[3].contents[0]
    reg_imdb = re.search(r'\d.\d\d', imdb)
    imdb = reg_imdb.group(0)
    tm = datetime.datetime.strftime(datetime.datetime.now(datetime.timezone.utc), '%Y-%m-%d %H:%M:%S')
    film = {'name': name, 'kp_rating': kp_rating, 'imdb_rating': imdb,
            'year': year, 'url': link, '_utc_timestamp': tm}
    yield film

2) (1 балл) Добавьте в функцию параметры, ограничивающие количество фильмов (чтобы вместо всего Топа-500 можно было скачать, например, 17 первых фильмов) и изменяющие время перерыва между запросами (чтобы вместо 0.5 с можно было задавать произвольное время в секундах). Добавьте проверку на корректность переданных параметров функции - если передано некорректное значение аргумента, нужно выбрасывать подходящее исключение.

In [0]:
def kino_gen(n :int, sec : Union[int, float]) -> Iterable[Dict[str, Any]]: 
  if n<0 or sec<0:
    raise ValueError('Допустимы только неотрицательные значения')
  if n>500:
    raise ValueError('Превышено максимальное значение n - 500')
  required_tags = []
  for page in [1,2,3]:
     url = 'https://www.kinopoisk.ru/top/lists/1/filtr/all/sort/order/perpage/200/page/{}/'.format(page)
     headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}  
     r = requests.get(url, headers=headers)
     html_doc = r.text  
     soup = BeautifulSoup(html_doc, 'html.parser')
     required_tags += soup.find_all('tr', ' js-film-list-item')
     time.sleep(sec)
  for x in range(n):
    td_blocks = required_tags[x].find_all('td')
    div_block1 = td_blocks[1].find('div')
    div_block2 = td_blocks[2].find('div', 'WidgetStars')
    divs_in_td2 = td_blocks[2].find_all('div')
    name = div_block1.find('a').contents[0]
    reg_year = re.search(r'(\d+)', div_block1.find('span').contents[0])
    if len(reg_year.group(1)) < 4:
      year = ''
    else:
      year = reg_year.group(1)
    kp_rating = div_block2.get('data-film-rating')
    link = 'https://www.kinopoisk.ru/film/%s/' % div_block2.get('data-film-id')
    imdb = divs_in_td2[3].contents[0]
    reg_imdb = re.search(r'\d.\d\d', imdb)
    imdb = reg_imdb.group(0)
    tm = datetime.datetime.strftime(datetime.datetime.now(datetime.timezone.utc), '%Y-%m-%d %H:%M:%S')
    film = {'name': name, 'kp_rating': kp_rating, 'imdb_rating': imdb,
            'year': year, 'url': link, '_utc_timestamp': tm}
    yield film

3) (0.5 балла) Выгрузите весь Топ-500 и превратите его в список словарей. Отсортируйте этот список по убыванию рейтинга IMDb (https://wiki.python.org/moin/HowTo/Sorting)

In [0]:
top500 = list(kino_gen())
top500 = sorted(top500, key=lambda film: film['imdb_rating'], reverse=True)

4) (0.5 балла) Используйте один из контейнеров из модуля [collections](https://docs.python.org/3/library/collections.html), чтобы собрать словарь вида "год выхода фильма -> количество фильмов из топ-500, вышедших в этот год". Выведите самый популярный год, а также минимальный и максимальный.

In [0]:
from collections import Counter

c = Counter()
for film in top500:
  try:
    c[int(film['year'])] +=1
  except:
    pass 
c = dict(c)

In [0]:
pop_year = ''
for k,v in c.items():
  if v == max(c.values()):
    pop_year = k

print('Самый популярный год: ', pop_year)

Самый популярный год:  2004


In [0]:
print('Максимальный год: ', max(c.keys()))
print('Минимальный год: ', min(c.keys()))

Максимальный год:  2019
Минимальный год:  1921


5) (1.5 балла) Напишите функцию, которая записывает ваш список словарей в файл в нужном формате, сортируя его по выбранному полю по возрастанию или убыванию. 

Проверяйте аргументы на корректность. Там, где возможно, используйте встроенные функции Python, вручную писать конвертацию в JSON не нужно. Не забывайте закрывать файл после окончания записи (https://stackoverflow.com/questions/25070854/why-should-i-close-files-in-python)


Возможные аргументы для `filetype`:
* `json` - https://docs.python.org/3.8/library/json.html
* `csv` - https://docs.python.org/3.8/library/csv.html
* `tsv` - то же, что CSV, но вместо запятых используются символы табуляции '\t')
* `jsonl` - каждая строка файла соответствует JSON-представлению какого-то из элементов массива. (http://jsonlines.org/examples/)

Обратите внимание, что вне зависимости от значений параметров `order_by` и `ascending`, исходный объект меняться **не должен**.

In [0]:
import json
import csv

def dump(film_list: List[Dict[str, Any]], path: str, filetype: str, order_by: Union[None, str] = None, ascending: bool = True) -> None:
  if filetype not in ['json', 'csv', 'tsv', 'jsonl']:
    raise ValueError('Невозможный аргумент для переменной filetype')
  if order_by not in film_list[0].keys():
    raise ValueError('Невозможный аргумент для переменной filetype')
  f = open(path, 'w')
  if filetype == 'json':
    f.write(json.dumps(sorted(film_list, key=lambda film: film[order_by], reverse=ascending), ensure_ascii = False))
  elif filetype=='csv':
    fieldnames = ['name', 'kp_rating', 'imdb_rating', 'year', 'url', '_utc_timestamp' ]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for film in sorted(film_list, key=lambda film: film[order_by], reverse=ascending):
      writer.writerow(film)
  elif filetype == 'tsv':
    fieldnames = 'name\tkp_rating\timdb_rating\tyear\turl\t_utc_timestamp'
    f.write(fieldnames)
    f.write('\n')
    for film in sorted(film_list, key=lambda film: film[order_by], reverse=ascending):
      row = '{}\t{}\t{}\t{}\t{}\t{}'.format(film['name'], film['kp_rating'], film['imdb_rating'],
                                            film['year'], film['url'], film['_utc_timestamp'])
      f.write(row)
      f.write('\n')
  elif filetype == 'jsonl':
    for film in sorted(film_list, key=lambda film: film[order_by], reverse=ascending):
      f.write(json.dumps(film, ensure_ascii = False))
      f.write('\n')
  f.close()
  



dump(top500, 'myfile.tsv', filetype='tsv', order_by='name', ascending=True)

## Часть 2. Вычисляем по IP

0. Почитайте, как устроены IPv4-адреса (https://en.wikipedia.org/wiki/IPv4). 
1. (0.5 балла) Напишите функцию, возвращающую строку, представлящую случайный IPv4-адрес (вам поможет модуль [random](https://docs.python.org/3.8/library/random.html)).
2. (0.5 балла) Сгенерируйте список из 100 **разных** IPv4-адресов.
3. (1 балл) С помощью https://freegeoip.app/ выгрузите для каждого из этих адресов информацию в формате JSON. Делайте случайные перерывы от 0.5 до 1 секунды между запросами (снова поможет модуль `random`).
4. (1.5 балла) В ответе этого API есть поле `time_zone`. Это часовой пояс местоположения, где теоретически находится сервер с таким IP.


* Сгенерируйте случайный момент времени в 2019 году (по UTC времени). 
* Выведите его в виде [unix timestamp](https://timestamp.online/)
* Приведите его ко всем встретившимся часовым поясам (см. [тут](https://docs.python.org/3/library/datetime.html?highlight=tzinfo#datetime.tzinfo)). Выведите результат.

In [0]:
import random

def random_ip() -> str:
  ipv4 = '{}.{}.{}.{}'.format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
  return ipv4

In [0]:
ip_list = []
while len(ip_list) < 100:
  element = random_ip()
  if element not in ip_list:
    ip_list.append(element)

In [0]:
responses = []
headers = {
    'accept': "application/json",
    'content-type': "application/json"}

for ip in ip_list:
  url = "https://freegeoip.app/json/"
  url = url + str(ip)
  response = requests.get(url, headers=headers)
  responses.append(response.json())
  time.sleep(random.uniform(0.5,1))

timezones = set([i['time_zone'] for i in responses])

In [0]:
timezones = set([i['time_zone'] for i in responses])
start_time= int(time.mktime(time.strptime("2019-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')))
end_time = int(time.mktime(time.strptime("2019-12-31 23:59:59", '%Y-%m-%d %H:%M:%S')))
random_moment = random.randint(start_time, end_time)
print(random_moment)

1555005183


In [0]:
import pytz

dt = datetime.datetime.fromtimestamp(random_moment)

for tz in timezones:
  try:
    timezone = pytz.timezone(tz)
    loc_dt = dt.astimezone(timezone)
    print(timezone, loc_dt.strftime('%Y-%m-%d %H:%M:%S'))
  except:
    pass

Asia/Bangkok 2019-04-12 00:53:03
America/Lima 2019-04-11 12:53:03
America/New_York 2019-04-11 13:53:03
America/Los_Angeles 2019-04-11 10:53:03
Europe/Ljubljana 2019-04-11 19:53:03
Europe/Copenhagen 2019-04-11 19:53:03
America/Chicago 2019-04-11 12:53:03
Australia/Brisbane 2019-04-12 03:53:03
America/Edmonton 2019-04-11 11:53:03
Europe/Rome 2019-04-11 19:53:03
Australia/Sydney 2019-04-12 03:53:03
America/Argentina/Buenos_Aires 2019-04-11 14:53:03
Africa/Cairo 2019-04-11 19:53:03
Europe/Oslo 2019-04-11 19:53:03
America/Sao_Paulo 2019-04-11 14:53:03
America/Toronto 2019-04-11 13:53:03
Asia/Shanghai 2019-04-12 01:53:03
Europe/Amsterdam 2019-04-11 19:53:03
Asia/Tokyo 2019-04-12 02:53:03
Europe/Moscow 2019-04-11 20:53:03
Australia/Melbourne 2019-04-12 03:53:03
Europe/London 2019-04-11 18:53:03
Europe/Berlin 2019-04-11 19:53:03
Asia/Tbilisi 2019-04-11 21:53:03
Asia/Kolkata 2019-04-11 23:23:03
Asia/Seoul 2019-04-12 02:53:03
Europe/Sofia 2019-04-11 20:53:03
Europe/Paris 2019-04-11 19:53:03
